In [17]:
import numpy as np
import matplotlib.pyplot as plt
from helpers import *
from cleaning_data import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
DATA_PATH = '/Users/williamjallot/Desktop/ML/dataset'
x_train, x_test, y_train, train_ids, test_ids, labels =  load_csv_data(DATA_PATH, sub_sample=False)
labels.pop(0) 

'Id'

In [18]:
x_train_cleaned = clean_data_x(x_train, labels)

    
np.savetxt("/Users/williamjallot/Desktop/ML/dataset/x_trained.cleaned.csv", x_train_cleaned, delimiter=",")

[[ 4.58333917  0.18042695 -0.0247581  ...  0.524848   -0.01449403
   0.38877291]
 [ 5.58333917  0.06044349 -0.0247581  ...  0.524848   -1.01449403
  -0.61122709]
 [ 3.58333917  0.66036077 -0.0247581  ... -0.475152   -0.01449403
   0.38877291]
 ...
 [ 3.58333917  0.66036077 -0.0247581  ...  0.524848    0.98550597
   0.38877291]
 [ 5.58333917  1.86019534 -0.0247581  ...  0.524848   -0.01449403
   0.38877291]
 [ 2.58333917 -0.29950688 -0.0247581  ... -0.475152   -0.01449403
   0.38877291]]
[[ 1.21992257e+01 -1.87909038e-01 -1.32948590e-01 ... -3.73757549e-02
  -2.15906882e-02 -8.93713691e-03]
 [-1.87909038e-01  1.00000305e+00 -4.58618085e-03 ...  3.59719460e-02
  -3.01946308e-02 -3.16935323e-02]
 [-1.32948590e-01 -4.58618085e-03  2.41452125e-02 ...  2.65704008e-03
  -1.94356432e-03 -2.16255813e-03]
 ...
 [-3.73757549e-02  3.59719460e-02  2.65704008e-03 ...  7.31863055e-01
  -2.36483354e-02 -1.96500818e-02]
 [-2.15906882e-02 -3.01946308e-02 -1.94356432e-03 ... -2.36483354e-02
   2.07150330